# Setup

In [102]:
from openai import OpenAI
import os, json, itertools
from IPython.display import display, Markdown

In [9]:
from google.colab import userdata

# Set OPENAI API KEY in the os
os.environ['OPENAI_API_KEY'] = userdata.get('ai_agents_openai')

In [10]:
client = OpenAI()

In [3]:
# Define model inputs
MODEL = "gpt-4.1"
MODEL_MINI = "gpt-4.1-mini"
TOOLS = [{"type": "web_search"}]
INCLUDE_RESULTS = ["web_search_call.results"]  # return raw search hits

In [4]:
# ---------- 1. get initial topic ----------
topic = input("Research topic: ").strip()

Research topic: Vibe coding


In [30]:
# ---------- 2. ask five clarifying questions ----------
developper_message = """You are a deep researcher"""
prompt = f"""
Ask 5 numbered clarifying questions to about: {topic}.
The goal of the questions is to understand the intended purpose of the research.
Reply only with the questions."""

clarify = client.responses.create(
    model=MODEL_MINI,
    input = prompt,
    instructions=developper_message,

)

In [31]:
# get the questions from the output and split by new line
questions = clarify.output[0].content[0].text.split("\n")
questions

['1. What specific aspects or features of "Vibe coding" are you aiming to explore or analyze in your research?  ',
 '2. Are you focusing on the technical implementation, user experience, or cultural impact of Vibe coding?  ',
 '3. What is the target audience or user group for the Vibe coding concept you are investigating?  ',
 '4. Are you interested in comparing Vibe coding to other coding paradigms or methodologies?  ',
 '5. What are the intended practical applications or outcomes you hope to achieve through this research on Vibe coding?']

In [36]:
# Ask each of the questions to the user
answers = []
for question in questions:
  # The built-in input() function can now be called correctly
  answer = input(f"{question}: ")
  answers.append(answer)

1. What specific aspects or features of "Vibe coding" are you aiming to explore or analyze in your research?  : how to vibe code and best practices
2. Are you focusing on the technical implementation, user experience, or cultural impact of Vibe coding?  : technical implementation
3. What is the target audience or user group for the Vibe coding concept you are investigating?  : non technical people
4. Are you interested in comparing Vibe coding to other coding paradigms or methodologies?  : no, just how to start vibe coding
5. What are the intended practical applications or outcomes you hope to achieve through this research on Vibe coding?: a guide to users


In [40]:
# Define input for the model
prompt_goals = f"""
Using the user answers {answers}, write a short goal sentence for the research about {topic}.
Output a JSON list of 5 web search queries that will reach it.
Format: {{\"goal\": \"...\", \"queries\": [\"q1\", ...]}}
"""

In [42]:
# ---------- 3. set goals + create first 5 searches ----------
goal_and_queries = client.responses.create(
    model=MODEL,
    previous_response_id=clarify.id,
    input=prompt_goals,
    instructions=
        developper_message,
)

In [43]:
goal_and_queries.output[0].content[0].text

'{\n  "goal": "To develop a user-friendly guide on how to start vibe coding and share best technical implementation practices specifically for non-technical people.",\n  "queries": [\n    "how to start vibe coding for beginners",\n    "vibe coding best practices for non technical users",\n    "step by step technical guide to vibe coding",\n    "vibe coding easy implementation for non programmers",\n    "vibe coding tutorial for complete beginners"\n  ]\n}'

In [45]:
plan = json.loads(goal_and_queries.output[0].content[0].text)
plan

{'goal': 'To develop a user-friendly guide on how to start vibe coding and share best technical implementation practices specifically for non-technical people.',
 'queries': ['how to start vibe coding for beginners',
  'vibe coding best practices for non technical users',
  'step by step technical guide to vibe coding',
  'vibe coding easy implementation for non programmers',
  'vibe coding tutorial for complete beginners']}

In [44]:
goal = plan["goal"]
queries = plan["queries"]

In [47]:
queries[0]

'how to start vibe coding for beginners'

# Web Search

In [50]:
resp = client.responses.create(
        model=MODEL,
        input=f"Search: {queries[0]}",
        tools=TOOLS,
        include=INCLUDE_RESULTS,
    )

In [69]:
resp.output[0].id

'ws_68270340d9dc8198b0288470dcd5b48a047101d773d541f7'

In [86]:
resp.output[1].content[0].annotations

[AnnotationURLCitation(end_index=558, start_index=477, title='Vibe coding', type='url_citation', url='https://en.wikipedia.org/wiki/Vibe_coding?utm_source=openai'),
 AnnotationURLCitation(end_index=957, start_index=855, title='Vibe Coding Explained: A Creative Approach to Programming | Medium', type='url_citation', url='https://medium.com/%40swengcrunch/vibe-coding-explained-26973e7797e0?utm_source=openai'),
 AnnotationURLCitation(end_index=1477, start_index=1353, title='The Best Guide to Vibe Coding: Create Software Without Coding Skills', type='url_citation', url='https://www.teapowered.pro/insights/guides/vibe-coding-writing-code-for-non-devs-risks?utm_source=openai'),
 AnnotationURLCitation(end_index=1732, start_index=1608, title='The Best Guide to Vibe Coding: Create Software Without Coding Skills', type='url_citation', url='https://www.teapowered.pro/insights/guides/vibe-coding-writing-code-for-non-devs-risks?utm_source=openai'),
 AnnotationURLCitation(end_index=1954, start_index

In [87]:
def run_search(q):
    resp = client.responses.create(
        model=MODEL,
        input=f"Search: {q}",
        tools=TOOLS,
        include=INCLUDE_RESULTS,
    )
    return {"query": q,
            "resp_id": resp.output[0].id,
            "research_output": resp.output[1].content[0].text}

In [82]:
run_search(queries[0])

{'query': 'how to start vibe coding for beginners',
 'resp_id': 'ws_682707350c8081988f89fe0cc53947d80ecfe65425416f0a',
 'annotations': [AnnotationURLCitation(end_index=538, start_index=457, title='Vibe coding', type='url_citation', url='https://en.wikipedia.org/wiki/Vibe_coding?utm_source=openai'),
  AnnotationURLCitation(end_index=1166, start_index=1062, title="Getting Started with Vibe Coding: A Beginner's Guide to Full-Stack App Builders", type='url_citation', url='https://securevibecoding.io/getting-started-with-vibe-coding/?utm_source=openai'),
  AnnotationURLCitation(end_index=1583, start_index=1466, title='The beginner’s guide to vibe coding | by Nicolle Weeks | May, 2025 | Medium', type='url_citation', url='https://medium.com/%40nicolle.weeks/the-beginners-guide-to-vibe-coding-b5f1eaf0f3e5?utm_source=openai'),
  AnnotationURLCitation(end_index=1819, start_index=1695, title='The Best Guide to Vibe Coding: Create Software Without Coding Skills', type='url_citation', url='https://

In [88]:
def evaluate(collected):
    # Ask the model if we’re done. Return bool.
    review = client.responses.create(
        model=MODEL,
        input=[
            {"role": "developer", "content": f"Research goal: {goal}"},
            {"role": "assistant", "content": json.dumps(collected)},
            {"role": "user", "content":
                "Does this information fully satisfy the goal? "
                "Answer Yes or No only."}
        ],
        instructions=developper_message,
    )
    return "yes" in review.output[0].content[0].text.lower()

In [90]:
collected = []
for _ in itertools.count():                      # open-ended loop
    for q in queries:
        collected.append(run_search(q))
    if evaluate(collected):
        break

    # ask for 5 more searches
    more = client.responses.create(
        model=MODEL,
        input=[
            {"role": "assistant", "content": f"Current data: {json.dumps(collected)}"},
            {"role": "user",
             "content": ("We still haven’t met the goal. Give 5 more, new, "
                         "high-value web search queries only as a JSON list.")}
        ],
        previous_response_id=goal_and_queries.id
    )
    queries = json.loads(more.output[0].content[0].text)

In [103]:
# ---------- 5. final synthesis with citations ----------
final = client.responses.create(
    model=MODEL,
    input=[
        {"role": "developer", "content": (f"Write a complete answer that meets the goal: {goal}. "
                     "Cite sources inline using [n] and append a reference "
                     "list mapping [n] to url.")},
        {"role": "assistant", "content": json.dumps(collected)},
    ],
    instructions=developper_message,
)
print("\n=== FINAL REPORT ===\n")
display(Markdown(final.output[0].content[0].text))


=== FINAL REPORT ===



## User-Friendly Guide to Starting Vibe Coding & Best Implementation Practices for Non-Technical People

Vibe coding is an emerging way to build apps using natural language prompts and AI code assistants—making it possible for non-technical users to bring their software ideas to life. Rather than writing code yourself, you describe what you want the program to do, and AI helps turn your "vibe" into working software [1][2].

Here’s a step-by-step guide for getting started and key best practices for success:

---

### 1. Understand What Vibe Coding Is  
Vibe coding involves collaborating with an AI (like GitHub Copilot, Replit Ghostwriter, or the Cursor editor) by explaining your app’s behavior in English. The AI generates and improves the code based on your ongoing feedback until your idea comes to life [1][2].

---

### 2. Set Up Your Workspace  
- **Pick a Platform**: Use beginner-friendly, AI-powered tools such as:
  - **Replit** (cloud-based, easy for beginners) [3][4]
  - **Cursor** (AI-enhanced code editor) [1][4]
  - **GitHub Copilot** (works inside code editors) [1]
  - **Lovable.dev** (no-code/low-code, design-focused) [4]
- **Create a Free Account** (most tools offer a free plan).
- **Find a Distraction-Free Environment** to help you focus.

---

### 3. Shape Your Idea Clearly  
- **Describe Your App**: Write down in plain language what problem your app solves, its main features, and what a user does from start to finish [5].  
  *Example: “I want a simple app for listing groceries, with the ability to add, check off, and delete items.”*
- **List Key Functions**: Break big ideas into bite-sized tasks (e.g., “Add item,” “Check off item,” “Show all items”).

---

### 4. Start "Prompting" the AI  
- **Open Your Chosen Platform** and create a new project.
- **Type a Prompt** as if you’re talking to a helpful developer:
  *“Make a web app where the user can input a grocery item and see a list. Let them mark items as bought or remove them.”*
- **Be Specific**: The more details you give, the better the result [5].

---

### 5. Practice Best Prompting Techniques  
**Best Practices:**
- **Be Clear & Detailed:** Specify the goal, platform (web, mobile), and any must-have features [2][5].
- **Break Tasks Down:** Build step by step—first the layout, then the “add item” feature, then “mark as completed,” and so on [2][6].
- **Iterate Frequently:** Review what the AI generates, tell it what’s right/wrong, and ask for changes (e.g., “Please add a delete button for each item”) [7].
- **Use Examples:** If stuck, give sample inputs/outputs (e.g., “User enters ‘milk’; list shows ‘milk’”).

---

### 6. Build and Test in Small Steps  
- **Test After Each Step:** Try your app after each change. If something doesn’t work, ask the AI: “Why isn’t the delete button working?” or “Fix the error when adding an empty item.”
- **Don’t Fear Mistakes:** The process is iterative, and errors are learning moments.
- **Save Versions:** Many tools let you undo or save milestones—use them!

---

### 7. Share, Document & Learn  
- **Document as You Go:** Keep a simple description of what you asked and changes made. This makes it easier to backtrack or get help.
- **Engage with the Community:** Forums or Discord channels for your tool are great for advice and inspiration [2][7].
- **Deploy or Share:** Platforms like Replit let you launch your app with a custom link [6].

---

### 8. Key Tips to Succeed  
- **Start Small:** Simple projects like to-do lists, timers, or calculators are great practice [4].
- **Refine Your Prompts**: As you gain experience, you’ll get better at asking the AI for exactly what you want.
- **Seek Feedback:** Ask friends to try your app and suggest changes—describe those suggestions to the AI.

---

## References

[1] https://en.wikipedia.org/wiki/Vibe_coding  
[2] https://zencoder.ai/blog/vibe-coding-best-practices  
[3] https://docs.replit.com/tutorials/vibe-coding-101  
[4] https://www.datacamp.com/blog/vibe-coding  
[5] https://www.architjn.com/blog/vibe-coding-for-non-technical-users  
[6] https://securevibecoding.io/getting-started-with-vibe-coding  
[7] https://www.techiediaries.com/vibe-coding-programming-by-prompt/

---

By following this simple, hands-on approach and using clear communication with the AI, non-technical people can confidently start building their own apps and learn best practices for effective and enjoyable vibe coding.

In [104]:
!pip freeze

absl-py==1.4.0
accelerate==1.6.0
aiohappyeyeballs==2.6.1
aiohttp==3.11.15
aiosignal==1.3.2
alabaster==1.0.0
albucore==0.0.24
albumentations==2.0.6
ale-py==0.11.0
altair==5.5.0
annotated-types==0.7.0
antlr4-python3-runtime==4.9.3
anyio==4.9.0
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.7.2
arviz==0.21.0
astropy==7.0.2
astropy-iers-data==0.2025.5.12.0.38.29
astunparse==1.6.3
atpublic==5.1
attrs==25.3.0
audioread==3.0.1
autograd==1.8.0
babel==2.17.0
backcall==0.2.0
backports.tarfile==1.2.0
beautifulsoup4==4.13.4
betterproto==2.0.0b6
bigframes==2.4.0
bigquery-magics==0.9.0
bleach==6.2.0
blinker==1.9.0
blis==1.3.0
blobfile==3.0.0
blosc2==3.3.2
bokeh==3.7.3
Bottleneck==1.4.2
bqplot==0.12.44
branca==0.8.1
build==1.2.2.post1
CacheControl==0.14.3
cachetools==5.5.2
catalogue==2.0.10
certifi==2025.4.26
cffi==1.17.1
chardet==5.2.0
charset-normalizer==3.4.2
chex==0.1.89
clarabel==0.10.0
click==8.2.0
cloudpathlib==0.21.0
cloudpickle==3.1.1
cmake==3.31.6
cmdstanpy==1.2.5
colorcet